In [2]:
import pandas as pd

df = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/articles.csv")
print(df.shape)
df.head()

(105542, 25)


,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [3]:
ohe_columns = []
total = 0

for col in df.columns:
    if df[col].dtype == "int64" and len(df[col].unique()) <= 500:
        ohe_columns.append(col)
        total += len(df[col].unique())
    
    print(col, df[col].dtype, len(df[col].unique()))
    
    
print("Columns to use:", ohe_columns)

article_id int64 105542
product_code int64 47224
prod_name object 45875
product_type_no int64 132
product_type_name object 131
product_group_name object 19
graphical_appearance_no int64 30
graphical_appearance_name object 30
colour_group_code int64 50
colour_group_name object 50
perceived_colour_value_id int64 8
perceived_colour_value_name object 8
perceived_colour_master_id int64 20
perceived_colour_master_name object 20
department_no int64 299
department_name object 250
index_code object 10
index_name object 10
index_group_no int64 5
index_group_name object 5
section_no int64 57
section_name object 56
garment_group_no int64 21
garment_group_name object 21
detail_desc object 43405
Columns to use: ['product_type_no', 'graphical_appearance_no', 'colour_group_code', 'perceived_colour_value_id', 'perceived_colour_master_id', 'department_no', 'index_group_no', 'section_no', 'garment_group_no']


In [4]:
V = pd.get_dummies(df[ohe_columns], columns=ohe_columns).values
V.shape

(105542, 622)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(min_df=3)
V_desc = tfidf.fit_transform(df["detail_desc"].fillna("nodesc"))
V_desc.shape

(105542, 2456)

In [6]:
from sklearn.decomposition import TruncatedSVD
import numpy as np


EMB_SIZE = 512

V = np.hstack([V.astype("float32"), V_desc.todense()])

svd = TruncatedSVD(n_components=EMB_SIZE, random_state=0)
svd.fit(V)
print("Explained variance ratio:", svd.explained_variance_ratio_.sum().item())

Explained variance ratio: 0.9650845407066022


In [7]:
V = svd.transform(V)
print(V.shape)

np.save("articles.npy", V)

(105542, 512)
